TODO:
- add holiday indicator

In [1]:
import datetime

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1668799821812_0004,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
BASE_INPUT_DIR = "s3://tvlp-ds-users/kendra-frederick/shopping-grid/agg-raw-data_poc_v2/*"
# df = spark.read.parquet(BASE_INPUT_DIR).filter(F.col("shop_date") >= 20221025)
df = spark.read.parquet(BASE_INPUT_DIR)
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------------+------------------+-----------------+----+---+-------------+--------+---------------+--------------------+-------------+------------+------------------+-------+------+----------+-----------------+-----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+------+---------+-------------------------+---------------------+---------+
| market|out_origin_airport|out_destination_airport|out_departure_date|in_departure_date| pcc|gds|point_of_sale|currency|out_origin_city|out_destination_city|out_num_stops|in_num_stops|constricted_search|out_cxr|in_cxr|round_trip|out_avail_all_009|out_avail_any_000|out_avail_any_001|out_avail_any_002|in_avail_all_009|in_avail_any_000|in_avail_any_001|in_avail_any_002|  fare|fare_rank|total_solution_counts_day|total_shop_counts_day|shop_date|
+-------+------------------+-----------------------+------------------+-----------------+----+---+----------

## add basic features, filter

In [3]:
def convert_dates(df, ow=False):
    df.registerTempTable("data")
    df = spark.sql("""
        SELECT *,
            TO_DATE(CAST(UNIX_TIMESTAMP(CAST(out_departure_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS outDeptDt_dt,
            TO_DATE(CAST(UNIX_TIMESTAMP(CAST(shop_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS shopDate_dt
        FROM data
    """)
    if not ow:
        df = spark.sql("""
            SELECT *,
                TO_DATE(CAST(UNIX_TIMESTAMP(CAST(in_departure_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS inDeptDt_dt,
            FROM data
        """)
    return df


def add_date_features(df, ow=False):
    df_mod = df.withColumn('days_til_dept', F.datediff(
                        F.col('outDeptDt_dt'), F.col('shopDate_dt'))
                    )
    if not ow:
        df_mod = df_mod.withColumn('stay_duration', F.datediff(
                        F.col('inDeptDt_dt'), F.col('outDeptDt_dt'))
                   )
    return df_mod


def add_dow_ind(df, ow=False):
    ## add in DOW indicators
    df_mod = (df
        .withColumn("dept_dt_dow_int", (F.date_format("outDeptDt_dt", "u") - 1).cast(T.IntegerType()))
        .withColumn("dept_dt_dow", F.date_format("outDeptDt_dt", "E"))
    )
    if not ow:
        df_mod = (df_mod
        .withColumn("ret_dt_dow_int", (F.date_format("inDeptDt_dt", "u") - 1).cast(T.IntegerType()))
        .withColumn("ret_dt_dow", F.date_format("inDeptDt_dt", "E"))
        )
    return df_mod


AIRPORT_LOOKUP_PATH = "s3://tvlp-ds-users/kendra-frederick/reference-data/AIRPORT.CSV"
def filter_pos(df, pos, currency):
    airport_df = spark.read.option("header", "true").csv(AIRPORT_LOOKUP_PATH)

    """Filters on pos and currency, and also O & D in pos country.
    
    Also enriches data with O & D city.
    """
    pos_df = df.filter(
        (F.col("point_of_sale") == pos) & (F.col("currency") == currency)
    )

    # filter on org or dest in country of POS
    pos_df = (pos_df
                .join(
                    airport_df.select('airport_code', 'country_code'),
                    on=[pos_df['out_origin_airport'] == airport_df['airport_code']])
                .withColumnRenamed("country_code", "origin_country")
                .drop("airport_code")
            )
    pos_df = (pos_df
                .join(
                    airport_df.select('airport_code', 'country_code'),
                    on=[pos_df['out_destination_airport'] == airport_df['airport_code']])
                .withColumnRenamed("country_code", "destination_country")
                .drop("airport_code")
            )

    pos_df = pos_df.filter(
        (F.col('origin_country') == pos) | (F.col("destination_country") == pos)
    )
    return pos_df

# new in version e
def filter_dtd(df, dtd_cutoff):
    df_filt = df.filter(F.col("days_til_dept") > dtd_cutoff)
    return df_filt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new in version f:

In [4]:
def get_date_range(date_start, date_stop):
    """Return a list of dates between `date_start` and `date_stop`, inclusive."""
    num_days = (date_stop - date_start).days
    date_list = [date_start + datetime.timedelta(days=x) for x in range(num_days + 1)]
    return date_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
TDAY_HOL = get_date_range(datetime.date(2022,11,23), datetime.date(2022,11,27))
XMAS_HOL = get_date_range(datetime.date(2022,12,16), datetime.date(2023,1,2))
HOLIDAYS = TDAY_HOL + XMAS_HOL

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
HOLIDAYS

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[datetime.date(2022, 11, 23), datetime.date(2022, 11, 24), datetime.date(2022, 11, 25), datetime.date(2022, 11, 26), datetime.date(2022, 11, 27), datetime.date(2022, 12, 16), datetime.date(2022, 12, 17), datetime.date(2022, 12, 18), datetime.date(2022, 12, 19), datetime.date(2022, 12, 20), datetime.date(2022, 12, 21), datetime.date(2022, 12, 22), datetime.date(2022, 12, 23), datetime.date(2022, 12, 24), datetime.date(2022, 12, 25), datetime.date(2022, 12, 26), datetime.date(2022, 12, 27), datetime.date(2022, 12, 28), datetime.date(2022, 12, 29), datetime.date(2022, 12, 30), datetime.date(2022, 12, 31), datetime.date(2023, 1, 1), datetime.date(2023, 1, 2)]

In [7]:
# new in version f
def add_holiday_ind(df):
    df_mod = df.withColumn("is_holiday", F.when(F.col("outDeptDt_dt").isin(HOLIDAYS), 1).otherwise(0))
    return df_mod

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Note: previous POS + currency filter plus limited markets must essentially do geo-filtering for us. But keep here for best practice
df_pos_filt = filter_pos(df, 'US', 'USD')

df_with_day_feats = convert_dates(df_pos_filt, ow=True)
df_with_day_feats = add_date_features(df_with_day_feats, ow=True)

df_with_hol = add_holiday_ind(df_with_day_feats)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
(df_with_hol
 .filter(F.col("outDeptDt_dt").between(datetime.date(2022,12,20), datetime.date(2023,1,3)))
 .select("out_departure_date", "is_holiday")
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------+
|out_departure_date|is_holiday|
+------------------+----------+
|          20221224|         1|
|          20221224|         1|
|          20221224|         1|
|          20221224|         1|
|          20221224|         1|
|          20221224|         1|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
|          20230103|         0|
+------------------+----------+
only showing top 20 rows

In [10]:
df_dtd_filt = filter_dtd(df_with_hol, 3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.count(), df_pos_filt.count(), df_dtd_filt.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(14546219, 14546219, 14071936)

In [11]:
df_dtd_filt.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['market', 'out_origin_airport', 'out_destination_airport', 'out_departure_date', 'in_departure_date', 'pcc', 'gds', 'point_of_sale', 'currency', 'out_origin_city', 'out_destination_city', 'out_num_stops', 'in_num_stops', 'constricted_search', 'out_cxr', 'in_cxr', 'round_trip', 'out_avail_all_009', 'out_avail_any_000', 'out_avail_any_001', 'out_avail_any_002', 'in_avail_all_009', 'in_avail_any_000', 'in_avail_any_001', 'in_avail_any_002', 'fare', 'fare_rank', 'total_solution_counts_day', 'total_shop_counts_day', 'shop_date', 'origin_country', 'destination_country', 'outDeptDt_dt', 'shopDate_dt', 'days_til_dept', 'is_holiday']

# train-test split

Spark's `TrainValidationSplit` is built to be used alongside a model. Which we aren't doing here / yet. So maybe this should be done in Pandas afterall?

Question: do we want to split now, on all data (includes multiple entries per shop day + market + travel dates), or by combo of  shop day + market + travel dates? Hmmm / gah. Probably the latter?

In [ ]:
# train, test = df_mod.randomSplit([0.9, 0.1], seed=19)

# train.count(), test.count()

# train.groupBy('round_trip').count().show()

# 952593 / (952593 + 6957040)

# test.groupBy('round_trip').count().show()

# 106146 / (106146 + 773118)

## focus on one-ways for now

In [12]:
# df_ow = train.filter(F.col("round_trip") == 0)
df_ow = df_dtd_filt.filter(F.col("round_trip") == 0).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# collapse to one entry per shop day + market + travel dates

In [13]:
# rename things to be less verbose
df_ow = (df_ow
         .withColumnRenamed("total_shop_counts_day", "shop_counts")
         .withColumnRenamed("total_solution_counts_day", "solution_counts")
        )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# =============================
# FEATURE ENGINEERING
# =============================
def eng_avail_features_out(df):
    # for now, collapse the 'any' features into one
    df_mod = (df
              .withColumn("out_avail_any_low", 
                          F.col("out_avail_any_000").cast('int')
                          .bitwiseOR(F.col("out_avail_any_001").cast('int'))
                          .bitwiseOR(F.col("out_avail_any_002").cast('int'))
                         )
              .withColumn("out_avail_all_max", F.col("out_avail_all_009").cast('int'))
             )
    return df_mod


def agg_by_fare_rank(df):
    """Aggregate by fare rank, so there is one row per fare rank.
    Also re-computes day/date-based features.
    Note: Availability features should have already been engineered.

    """
    # for now, take average of avail features. 
        # Considered taking `any` and `bit_and` equivalents. Out-of-the-box functions not
        # available in this version of spark, however.
        # The averages do capture this info, though:
            # if any out_avail_max == 1, average will be non-zero
            # if all out_avail_min == 1, average will be 1
    df_agg = (df
                 .groupBy(['market', 'out_departure_date', 'shop_date', 'fare_rank'])
                 .agg(
                     F.count("*").alias("num_itin"),
                     F.first("fare").alias("fare"),
                     F.collect_set("out_cxr").alias("out_cxrs"),
                     F.first("solution_counts").alias("solution_counts"),
                     F.first("shop_counts").alias("shop_counts"),
                     # F.any("out_avail_all_max").alias("out_avail_all_max"),
                     F.mean("out_avail_all_max").alias("avg_out_avail_max"),
                     # F.bit_and("out_avail_any_low").alias("out_avail_all_low"),
                     F.mean("out_avail_any_low").alias("avg_out_avail_low"),
                 )
                )

    # alt: do this on raw data & then either agg on these above
    df_agg = convert_dates(df_agg, ow=True)
    df_agg = add_date_features(df_agg, ow=True)
    df_agg = add_dow_ind(df_agg, ow=True)
    df_agg = add_holiday_ind(df_agg)
    return df_agg


def collapse_fare_rank_features(df):
    """Shift fare rank features, so we are left with a single row per search criteria.
    
    Collapse 2nd-lowest fare info into lowest-fare row.
    """
    w = (Window.partitionBy("market", "out_departure_date", "shop_date").orderBy("fare_rank"))
    df_mod = (df
                  # note: this temporarily pulls in 3rd-lowest fare to 2nd-lowest row
                  # but when filter on fare_rank == 1 below, these column names become
                  # accurate (could name them "next-lowest" instead of "fr2")
                .withColumn("fr2_fare", F.lead("fare").over(w))
                .withColumn("fr2_out_cxrs", F.lead("out_cxrs").over(w))
                .withColumn("fr2_avg_out_avail_max", F.lead("avg_out_avail_max").over(w))
                .withColumn("fr2_avg_out_avail_low", F.lead("avg_out_avail_low").over(w))
                .filter(F.col("fare_rank") == 1)
            )

    df_mod = (df_mod
                .withColumn("fr1_fr2_out_cxrs_same", F.col("out_cxrs") == F.col("fr2_out_cxrs"))
                .withColumn("fr1_fr2_out_cxrs_overlap", F.size(F.array_intersect(F.col("out_cxrs"), F.col("fr2_out_cxrs"))) > 0)
            )
    return df_mod


def target_encode(df):
    """Target-encode appropriate features.
    Features:
        - days until departure 
    Note: Data should contain a single row per search criteria -- i.e. 'fare'
        column should correspond to the lowest fare (e.g. the target).
    """
    # target-encode by dtd
    w3 = (Window
        .partitionBy("market", "days_til_dept")
        .orderBy("shop_date")
        .rowsBetween(Window.unboundedPreceding, -1)
        )
    df_mod = (df
              .withColumn("avg_fare_dtd", F.mean("fare").over(w3))
             )
    return df_mod

def eng_shifted_day_features(df):
    """Engineer 'shifted' features based on shop and travel days.
    """
    shifted_features_dict = {
        "shop_day": {
            'sort_col': 'shop_date',
            'groupby_cols': ['market', 'out_departure_date'], #, 'inDeptDt'],
            'target_cols': ['fare', 'fr2_fare', 
                            'avg_out_avail_max', 'avg_out_avail_low',
                            # new in version e:
                            'num_itin', 'shop_counts', 'solution_counts']
        },        
        "dept_day": {
            'sort_col': 'out_departure_date',
            'groupby_cols': ['market', 'shop_date'],
            'target_cols': ['fare']
        },
    }
    df_mod = df.select("*")
    for feature, kwargs in shifted_features_dict.items():
        w = (Window
            .orderBy(kwargs['sort_col'])
            .partitionBy(kwargs['groupby_cols'])
            )

        for target_col in kwargs['target_cols']:
            df_mod = (df_mod
                .withColumn("{}_prev_{}".format(target_col, feature), F.lag(target_col).over(w))
                )
            if feature != 'shop_day':
                df_mod = (df_mod
                .withColumn("{}_next_{}".format(target_col, feature), F.lead(target_col).over(w))
                )
    # calc diff for dept day fares
    df_mod = (df_mod
              .withColumn('fare_diff_prev_dept_day', F.col('fare') - F.col('fare_prev_dept_day'))
              .withColumn('fare_diff_next_dept_day', F.col('fare') - F.col('fare_next_dept_day'))
    )
    return df_mod


# new in version e:
def eng_trailing_avg_features(df):
    # 3-day trailing avg of counts and min fare
    w = (Window
         .orderBy("shop_date")
         .partitionBy("market", "out_departure_date")
         .rowsBetween(-4, -1)
        )
    df_mod = (df
              .withColumn("trailing_avg_shop_counts", F.mean("shop_counts").over(w))
              .withColumn("trailing_avg_solution_counts", F.mean("solution_counts").over(w))
              .withColumn("trailing_avg_fare", F.mean("fare").over(w))
              .withColumn("trailing_std_fare", F.stddev("fare").over(w))
             )
    
    # Trailing average (unbounded) of fare diff between prev/next dept day
    days_to_sec = lambda i: i * 24 * 60 *60

    w2 = (Window
         .orderBy(F.col("shopDate_dt").cast("timestamp").cast("long"))
         .partitionBy("market", "dept_dt_dow")
         .rangeBetween(Window.unboundedPreceding, -days_to_sec(1))
        )
    df_mod = (df_mod
              .withColumn("avg_diff_prev_dept_day", F.mean("fare_diff_prev_dept_day").over(w2))
              .withColumn("avg_diff_next_dept_day", F.mean("fare_diff_next_dept_day").over(w2))
             )
    
    # Calc estimated fare from fare & TA of fare diff
    df_mod = (df_mod
              .withColumn("est_fare_from_prev_dept_day",
                          F.col("fare_prev_dept_day") + F.col("avg_diff_prev_dept_day")
                         )
              .withColumn("est_fare_from_next_dept_day",
                          F.col("fare_next_dept_day") + F.col("avg_diff_next_dept_day")
                         )
             )
          
    return df_mod

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_ow = eng_avail_features_out(df_ow)
df_ow_agg = agg_by_fare_rank(df_ow)
df_ow_mf = collapse_fare_rank_features(df_ow_agg)
df_ow_mf = target_encode(df_ow_mf)
df_ow_shift = eng_shifted_day_features(df_ow_mf)
df_ow_trail = eng_trailing_avg_features(df_ow_shift)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Check

In [17]:
df_ow_trail.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['market', 'out_departure_date', 'shop_date', 'fare_rank', 'num_itin', 'fare', 'out_cxrs', 'solution_counts', 'shop_counts', 'avg_out_avail_max', 'avg_out_avail_low', 'outDeptDt_dt', 'shopDate_dt', 'days_til_dept', 'dept_dt_dow_int', 'dept_dt_dow', 'is_holiday', 'fr2_fare', 'fr2_out_cxrs', 'fr2_avg_out_avail_max', 'fr2_avg_out_avail_low', 'fr1_fr2_out_cxrs_same', 'fr1_fr2_out_cxrs_overlap', 'avg_fare_dtd', 'fare_prev_shop_day', 'fr2_fare_prev_shop_day', 'avg_out_avail_max_prev_shop_day', 'avg_out_avail_low_prev_shop_day', 'num_itin_prev_shop_day', 'shop_counts_prev_shop_day', 'solution_counts_prev_shop_day', 'fare_prev_dept_day', 'fare_next_dept_day', 'fare_diff_prev_dept_day', 'fare_diff_next_dept_day', 'trailing_avg_shop_counts', 'trailing_avg_solution_counts', 'trailing_avg_fare', 'trailing_std_fare', 'avg_diff_prev_dept_day', 'avg_diff_next_dept_day', 'est_fare_from_prev_dept_day', 'est_fare_from_next_dept_day']

In [12]:
m = 'LGA-MIA'
dd = 20221110
# sd = 20221101
ex_df_mf = df_ow_trail.filter((F.col("market") == m) & (F.col("out_departure_date") == dd)) #& (F.col("shop_date") == sd))
(ex_df_mf.select('out_departure_date', 'shop_date', 'days_til_dept', 'fare', 
                 'fare_prev_shop_day', 'fr2_fare', 'fr2_fare_prev_shop_day',)
 .orderBy("shop_date")
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------+-------------+------+------------------+--------+----------------------+
|out_departure_date|shop_date|days_til_dept|  fare|fare_prev_shop_day|fr2_fare|fr2_fare_prev_shop_day|
+------------------+---------+-------------+------+------------------+--------+----------------------+
|          20221110| 20221015|           26| 43.98|              null|   48.59|                  null|
|          20221110| 20221016|           25| 43.98|             43.98|   48.59|                 48.59|
|          20221110| 20221017|           24| 42.98|             43.98|   43.98|                 48.59|
|          20221110| 20221018|           23| 42.98|             42.98|   48.59|                 43.98|
|          20221110| 20221019|           22| 42.98|             42.98|   47.98|                 48.59|
|          20221110| 20221020|           21| 47.98|             42.98|   48.59|                 47.98|
|          20221110| 20221021|           20| 47.98|             47.98|   

In [15]:
m = 'LGA-MIA'
dd = 20221110
# sd = 20221101
ex_df_mf = df_ow_trail.filter((F.col("market") == m) & (F.col("out_departure_date") == dd)) #& (F.col("shop_date") == sd))
(ex_df_mf.select('out_departure_date', 'shop_date', 'days_til_dept', 
                 'fare', 'avg_fare_dtd', 
                 'fare_prev_shop_day', 'trailing_avg_fare', 'trailing_std_fare',)
 .orderBy("shop_date")
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------+-------------+------+------------------+------------------+------------------+------------------+
|out_departure_date|shop_date|days_til_dept|  fare|      avg_fare_dtd|fare_prev_shop_day| trailing_avg_fare| trailing_std_fare|
+------------------+---------+-------------+------+------------------+------------------+------------------+------------------+
|          20221110| 20221015|           26| 43.98|              null|              null|              null|              null|
|          20221110| 20221016|           25| 43.98|             35.97|             43.98|             43.98|               NaN|
|          20221110| 20221017|           24| 42.98|             35.97|             43.98|             43.98|               0.0|
|          20221110| 20221018|           23| 42.98| 38.63999999999999|             42.98| 43.64666666666667|0.5773502691896258|
|          20221110| 20221019|           22| 42.98|           42.7275|             42.98|             43

In [ ]:
m = 'LGA-MIA'
dd = 20221110
# sd = 20221101
ex_df_mf = df_ow_trail.filter((F.col("market") == m) & (F.col("out_departure_date") == dd)) #& (F.col("shop_date") == sd))
(ex_df_mf.select('out_departure_date', 'shop_date', 'days_til_dept', 
                 'shop_counts', 'shop_counts_prev_shop_day', 'trailing_avg_shop_counts',
                 'solution_counts','solution_counts_prev_shop_day','trailing_avg_solution_counts',
                )
 .orderBy("shop_date")
 .show()
)

MIght want to clip off first X days, X being window size. Trailing avg is getting computed for less than X.

In [16]:
m = 'LGA-MIA'
# dd = 20221110
sd = 20221101
ex_df_mf = df_ow_trail.filter((F.col("market") == m) & (F.col("shop_date") == sd)) # (F.col("out_departure_date") == dd)
(ex_df_mf.select('out_departure_date', 'dept_dt_dow', 'fare', 
                 'fare_prev_dept_day', 'fare_diff_prev_dept_day', #'avg_diff_prev_dept_day', 'est_fare_from_prev_dept_day',
                 'fare_next_dept_day', 'fare_diff_next_dept_day', #'avg_diff_next_dept_day', 'est_fare_from_next_dept_day',
                )
 .orderBy("out_departure_date")
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------+-----+------------------+-----------------------+------------------+-----------------------+
|out_departure_date|dept_dt_dow| fare|fare_prev_dept_day|fare_diff_prev_dept_day|fare_next_dept_day|fare_diff_next_dept_day|
+------------------+-----------+-----+------------------+-----------------------+------------------+-----------------------+
|          20221105|        Sat|63.99|              null|                   null|             73.98|     -9.990000000000002|
|          20221106|        Sun|73.98|             63.99|      9.990000000000002|             47.98|     26.000000000000007|
|          20221107|        Mon|47.98|             73.98|    -26.000000000000007|             43.98|                    4.0|
|          20221108|        Tue|43.98|             47.98|                   -4.0|             47.98|                   -4.0|
|          20221109|        Wed|47.98|             43.98|                    4.0|             47.98|                    0.0|


Need to clip out dtd < 4

In [18]:
m = 'LGA-MIA'
# dd = 20221110
ddow = "Wed"
sd = 20221101
ex_df_mf = df_ow_trail.filter((F.col("market") == m) 
                              & (F.col("shop_date") == sd)
                              & (F.col("dept_dt_dow") == ddow)
                             )
(ex_df_mf.select('out_departure_date', 'dept_dt_dow', 'shop_date', 'fare', 
                 'fare_prev_dept_day', 'avg_diff_prev_dept_day', 'est_fare_from_prev_dept_day',
                 # 'fare_next_dept_day', 'avg_diff_next_dept_day', 'est_fare_from_next_dept_day',
                )
 .orderBy("shop_date", 'out_departure_date')
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------+---------+-----+------------------+----------------------+---------------------------+
|out_departure_date|dept_dt_dow|shop_date| fare|fare_prev_dept_day|avg_diff_prev_dept_day|est_fare_from_prev_dept_day|
+------------------+-----------+---------+-----+------------------+----------------------+---------------------------+
|          20221109|        Wed| 20221101|47.98|             43.98|     4.297050610820246|          48.27705061082024|
|          20221116|        Wed| 20221101|43.98|             43.98|     4.297050610820246|          48.27705061082024|
|          20221123|        Wed| 20221101|63.99|             43.98|     4.297050610820246|          48.27705061082024|
|          20221130|        Wed| 20221101|43.98|             43.98|     4.297050610820246|          48.27705061082024|
|          20221207|        Wed| 20221101|43.98|             43.98|     4.297050610820246|          48.27705061082024|
|          20221214|        Wed| 20221101|43.98|

In [19]:
m = 'LGA-MIA'
# dd = 20221110
ddow = "Thu"
sd = 20221101
ex_df_mf = df_ow_trail.filter((F.col("market") == m) 
                              & (F.col("shop_date") == sd)
                              & (F.col("dept_dt_dow") == ddow)
                             )
(ex_df_mf.select('out_departure_date', 'dept_dt_dow', 'shop_date', 'fare', 
                 # 'fare_prev_dept_day', ' 'avg_diff_prev_dept_day', 'est_fare_from_prev_dept_day',
                 'fare_next_dept_day', 'avg_diff_next_dept_day', 'est_fare_from_next_dept_day',
                )
 .orderBy("shop_date", 'out_departure_date')
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------+---------+------+------------------+----------------------+---------------------------+
|out_departure_date|dept_dt_dow|shop_date|  fare|fare_next_dept_day|avg_diff_next_dept_day|est_fare_from_next_dept_day|
+------------------+-----------+---------+------+------------------+----------------------+---------------------------+
|          20221110|        Thu| 20221101| 47.98|             47.98|   -15.716934673366838|          32.26306532663316|
|          20221117|        Thu| 20221101| 43.98|             47.98|   -15.716934673366838|          32.26306532663316|
|          20221124|        Thu| 20221101| 43.98|             47.98|   -15.716934673366838|          32.26306532663316|
|          20221201|        Thu| 20221101| 43.98|             47.98|   -15.716934673366838|          32.26306532663316|
|          20221208|        Thu| 20221101| 43.98|             47.98|   -15.716934673366838|          32.26306532663316|
|          20221215|        Thu| 2022110

# save

In [20]:
df_ow_trail.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

133001

In [18]:
df_ow_trail.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- market: string (nullable = true)
 |-- out_departure_date: long (nullable = true)
 |-- shop_date: long (nullable = true)
 |-- fare_rank: integer (nullable = true)
 |-- num_itin: long (nullable = false)
 |-- fare: double (nullable = true)
 |-- out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- solution_counts: long (nullable = true)
 |-- shop_counts: long (nullable = true)
 |-- avg_out_avail_max: double (nullable = true)
 |-- avg_out_avail_low: double (nullable = true)
 |-- outDeptDt_dt: date (nullable = true)
 |-- shopDate_dt: date (nullable = true)
 |-- days_til_dept: integer (nullable = true)
 |-- dept_dt_dow_int: integer (nullable = true)
 |-- dept_dt_dow: string (nullable = true)
 |-- is_holiday: integer (nullable = false)
 |-- fr2_fare: double (nullable = true)
 |-- fr2_out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fr2_avg_out_avail_max: double (nullable = true)
 |-- fr2_avg_out_avail_low: double

In [26]:
df_ow_trail.select(F.max("shop_date")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|max(shop_date)|
+--------------+
|      20221117|
+--------------+

In [19]:
last_shop_day = "20221117"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# named for the last shop day in data set
df_ow_trail.coalesce(1).write.mode("overwrite").parquet(f"s3://tvlp-ds-users/kendra-frederick/shopping-grid/engineered-features/parquet/{last_shop_day}/") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# drop arrays and save as a csv, for download & local analysis
(df_ow_trail
 .drop('out_cxrs', 'fr2_out_cxrs')
 .coalesce(1)
 .write.mode("overwrite")
 .csv(f"s3://tvlp-ds-users/kendra-frederick/shopping-grid/engineered-features/csv/{last_shop_day}/", header=True)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…